In [1]:
import requests

In [ ]:
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(f"{base_url}/v3/language", headers=headers)
assert response.status_code == 200
assert response.json() == [{"iso639": "eng", "name": "english"}]